In [1]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

Merge the 3 files into 1 file

In [2]:
import glob

read_files = glob.glob("data/*.txt")

with open("data/sentences_file_result.txt", "wb") as outfile:
    for f in read_files:
        with open(f, "rb") as infile:
            outfile.write(infile.read())

Transformation

In [14]:
file_path = 'data/sentences_file_result.txt'

data = []
with open(file_path, 'r') as file:
    for line in file:
        parts = line.split(' ', 1)
        if len(parts) == 2:
            number, text = parts
            data.append((number, text.strip()))

df = pd.DataFrame(data, columns=['label', 'text'])

df

,label,text
0,1,I liked product and quality .
1,1,Nice look and super product .
2,1,Value for money .
3,1,Good watch all are ok in this price but one is...
4,1,So far i liked the most is the battery capacit...
...,...,...
333,1,The best cinematic meditation on greed since '...
334,1,Classic .
335,0,The world is not black and white .
336,1,Wonderful .


In [15]:
df_clean = df.drop_duplicates()

df_clean

,label,text
0,1,I liked product and quality .
1,1,Nice look and super product .
2,1,Value for money .
3,1,Good watch all are ok in this price but one is...
4,1,So far i liked the most is the battery capacit...
...,...,...
333,1,The best cinematic meditation on greed since '...
334,1,Classic .
335,0,The world is not black and white .
336,1,Wonderful .


In [16]:
def classify_sentence(label):
    if label is None:
        return "unknown"

    if label == 0:
        return "bad review"
    elif label == 1:
        return "good review"
    else:
        return "unknown"

In [18]:
# %pip install spark-nlp
# %pip install pyspark

&Load

In [19]:
import sparknlp
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, DoubleType, IntegerType
from pyspark.sql.functions import concat, col
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [20]:
# Initialize Spark
spark = SparkSession.builder.appName("SentenceClassifier").getOrCreate()
sparknlp.start()

# Register the UDF
classify_udf = udf(classify_sentence, StringType())

# Create a Spark DataFrame
spark_df = spark.createDataFrame(df_clean)

# Apply the UDF to classify the sentences and add it to a new column "classified_label"
classified_df = spark_df.withColumn("classified_label", classify_udf(spark_df["label"]))

# Save the combined DataFrame to a text file
classified_df.write.mode("overwrite").format("csv").save("output")

# Convert StringType to DoubleType for the "classified_label" and "label" columns
classified_df_sp = classified_df.withColumn("classified_label", classified_df["classified_label"].cast(DoubleType()))
classified_df_sp = classified_df.withColumn("label", classified_df["label"].cast(DoubleType()))

# Filter out rows with null values in "classified_label" or "label"
classified_df_spa = classified_df_sp.filter(col("classified_label").isNotNull() & col("label").isNotNull())

# Calculate accuracy using MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(predictionCol="label", labelCol="label", metricName="accuracy")
accuracy = evaluator.evaluate(classified_df_spa)

print("Accuracy:", accuracy)

# Stop the Spark session
spark.stop()


Accuracy: 1.0


In [24]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("example").getOrCreate()
spark_version = spark.version
print("Apache Spark version:", spark_version)
spark.stop()

Apache Spark version: 3.5.0


In [25]:
sparknlp.version()

'5.1.4'